In [1]:
from datetime import datetime
import requests
import pandas as pd
import csv
from fuzzywuzzy import process,fuzz

from io import StringIO
pd.set_option('display.max_columns', 500)


In [2]:
mlb_teams = {
    'ARI': 'Arizona Diamondbacks',
    'ATL': 'Atlanta Braves', 
    'BAL': 'Baltimore Orioles',
    'BOS': 'Boston Red Sox',
    'CHC': 'Chicago Cubs',
    'CWS': 'Chicago White Sox', 
    'CIN': 'Cincinnati Reds',
    'CLE': 'Cleveland Guardians',
    'COL': 'Colorado Rockies',
    'DET': 'Detroit Tigers',
    'HOU': 'Houston Astros',
    'KC': 'Kansas City Royals',
    'LAA': 'Los Angeles Angels',
    'LAD': 'Los Angeles Dodgers',
    'MIA': 'Miami Marlins',
    'MIL': 'Milwaukee Brewers',
    'MIN': 'Minnesota Twins', 
    'NYM': 'New York Mets',
    'NYY': 'New York Yankees',
    'OAK': 'Oakland Athletics',
    'PHI': 'Philadelphia Phillies',
    'PIT': 'Pittsburgh Pirates',
    'SD': 'San Diego Padres',
    'SF': 'San Francisco Giants',
    'SEA': 'Seattle Mariners',
    'STL': 'St. Louis Cardinals',
    'TB': 'Tampa Bay Rays',
    'TEX': 'Texas Rangers',
    'TOR': 'Toronto Blue Jays',
    'WSH': 'Washington Nationals'
}

In [3]:
batter_boxscores = pd.read_csv('datasets/complex_batters.csv')
pitcher_boxscores = pd.read_csv('datasets/complex_pitchers.csv')
batter_boxscores = batter_boxscores.rename(columns={
    'h' : 'Hits',
    'Batter Fantasy Score' : 'Hitter Fantasy Score',
    'k' : 'Hitter Strikeouts',
    'hr' : 'Home Runs',
    'rbi' : 'RBIs',
    'r' : 'Runs',
    'singles' : 'Singles',
    'TB' : 'Total Bases',
    'doubles' : 'Doubles',
    'H+R+RBIs' : 'Hits+Runs+RBIs',
    'bb': 'Walks',
    'sb' : 'Stolen Bases'
})

/var/folders/7f/nl87050s61973ky11lnw6gyr0000gn/T/ipykernel_8867/1868897738.py:1: DtypeWarning: Columns (40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  batter_boxscores = pd.read_csv('datasets/complex_batters.csv')
/var/folders/7f/nl87050s61973ky11lnw6gyr0000gn/T/ipykernel_8867/1868897738.py:2: DtypeWarning: Columns (12,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  pitcher_boxscores = pd.read_csv('datasets/complex_pitchers.csv')


In [4]:
url = 'http://192.168.1.200/data/live_prop_data.csv'


response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    data = response.content.decode('utf-8')
else:
    print(f"Failed to retrieve data: status code {response.status_code}")
    data = None
if data:
    # Convert string data to StringIO object
    data_io = StringIO(data)
    # Create DataFrame
    live_df = pd.read_csv(data_io)

In [5]:
url = 'http://192.168.1.200/data/data.csv'


response = requests.get(url)
# Check if the request was successful
if response.status_code == 200:
    data = response.content.decode('utf-8')
else:
    print(f"Failed to retrieve data: status code {response.status_code}")
    data = None
if data:
    # Convert string data to StringIO object
    data_io = StringIO(data)
    # Create DataFrame
    props_df = pd.read_csv(data_io)

In [6]:
current_date = datetime.now().strftime('%Y-%m-%d')

current_day_props = props_df[props_df['game_date'] == current_date]

pitcher_props = current_day_props[current_day_props['position'].isin(['P','SP'])]
batter_props = current_day_props[~current_day_props['position'].isin(['P', 'SP'])]

In [7]:
batter_props = batter_props.sort_values(by=['Name','prop_type','odds_type'])
batter_props = batter_props.drop_duplicates()

In [8]:

updated_names = []

for name in batter_props['Name'].unique():
    if name in batter_boxscores['Name'].to_list():
        updated_names.append(name)

In [16]:
organized_prop_df = []

for player_name in updated_names:
    player_props = batter_props[batter_props['Name'] == player_name]
    player_boxscore = batter_boxscores[batter_boxscores['Name'] == player_name][::-1]

    for index, row in player_props.iterrows():

        prop_dict = {
            'Team' : player_boxscore['Team'].iloc[0],
            'game_date' : row['game_date'],
            'Name' : row['Name'],
            'prop_line' : row['prop_line'],
            'odds_type' : row['odds_type'],
            'prop_type' : row['prop_type'],
            'PP_ID' : row['PP_ID'],
            'statsAPI_ID' : player_boxscore['personId'].iloc[0],
            'Hit Rate' : 0,
            'Push Rate' : 0,
            'Hit' : 0,
            'Push' : 0,
            'Game Size' : 0
        }

        stat_list = player_boxscore[row['prop_type']].to_list()

        actual_num_of_games = 20

        number_of_games = actual_num_of_games

        if len(stat_list) < (number_of_games + 1):
            number_of_games = len(stat_list)
        else:
            stat_list = stat_list[:number_of_games]
        
        prop_dict['Game Size'] = number_of_games
        
        prop_dict['Hit'] = len([num for num in stat_list if num > row['prop_line']])
        prop_dict['Push'] = len([num for num in stat_list if num == row['prop_line']])
        prop_dict['Hit Rate'] = f"{round(prop_dict['Hit'] / prop_dict['Game Size'] * 100,2)}%"
        prop_dict['Push Rate'] = f"{round(prop_dict['Push'] / prop_dict['Game Size'] * 100,2)}%"

        for i in range(1, actual_num_of_games+1):
            try:
                prop_dict[str(i)] = stat_list[i-1]
            except:
                prop_dict[str(i)] = None




        organized_prop_df.append(prop_dict)
    

In [17]:
organized_prop_df = pd.DataFrame(organized_prop_df)
#player_boxscore


In [18]:
organized_prop_df.to_excel('Prop_Table.xlsx', index=False)